### This is my work in progress exploring this dataset. 
#### Brian of London

In [140]:
import pandas as pd
import csv
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

In [141]:
df = pd.read_csv('data/nyt_ts.csv', index_col='timestamp')
states = df['state'].unique()

In [142]:
df['votes_prev'] = df.groupby('state')['votes'].shift(1)
df['votes_diff'] = [v - vp for v,vp in zip(df['votes'],df['votes_prev'])]
df.fillna(0, inplace=True)


In [220]:
# Calculate absolute vote values per time stamp
df['trump_v'] = [(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes_diff'])]
df['biden_v'] = [(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes_diff'])]
df['trd_v']= [(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes_diff'])]
df['check_v'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_v'],df['biden_v'],df['trd_v'],df['votes_diff'])]

df['trump_c'] = [(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes'])]
df['biden_c'] = [(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes'])]
df['trd_c']= [(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes'])]
df['check_c'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_c'],df['biden_c'],df['trd_c'],df['votes'])]

# check_v is showing rounding errors
df.groupby('state').last()

,Unnamed: 0,votes,eevp,eevp_source,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,...,votes_prev,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c
state,,,,,,,,,,,,,,,,,,,,,
alabama,179,2309900,99,edison,2300000,1318255,2074338,2123372,0.621,0.365,...,2297216.0,12684.0,7876.764,4629.660,0.0,177.576,1434447.900,8.431135e+05,0,32338.600
alaska,5,240234,75,edison,367000,163387,300495,318608,0.580,0.381,...,190970.0,49264.0,28573.120,18769.584,0.0,1921.296,139335.720,9.152915e+04,0,9369.126
arizona,429,3348758,98,edison,3400000,1252401,2306559,2604657,0.490,0.494,...,3343467.0,5291.0,2592.590,2613.754,0.0,84.656,1640891.420,1.654286e+06,0,53580.128
arkansas,319,1209372,99,edison,1225000,684872,1069468,1130635,0.626,0.346,...,1209360.0,12.0,7.512,4.152,0.0,0.336,757066.872,4.184427e+05,0,33862.416
california,600,15975229,95,edison,16800000,4483810,13038547,14237884,0.336,0.643,...,15937716.0,37513.0,12604.368,24120.859,0.0,787.773,5367676.944,1.027207e+07,0,335479.809
colorado,716,3212169,97,edison,3300000,1202484,2569520,2780247,0.419,0.554,...,3173957.0,38212.0,16010.828,21169.448,0.0,1031.724,1345898.811,1.779542e+06,0,86728.563
connecticut,941,1821878,98,edison,1850000,673215,1558993,1644920,0.392,0.592,...,1821824.0,54.0,21.168,31.968,0.0,0.864,714176.176,1.078552e+06,0,29150.048
delaware,952,504010,99,edison,510000,185127,413921,443814,0.398,0.588,...,502392.0,1618.0,643.964,951.384,0.0,22.652,200595.980,2.963579e+05,0,7056.140
florida,1237,11075446,99,edison,11100000,4617886,8474179,9501617,0.512,0.478,...,11075806.0,-360.0,-184.320,-172.080,-0.0,-3.600,5670628.352,5.294063e+06,0,110754.460


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [188]:
# Calculate the ratio of D/R
filt = (df['trump_v'] == 0) | (df['biden_v'] == 0)
dfn = df[~filt]

dfn['r_to_d'] = [r / d for r,d in zip(dfn['trump_v'],dfn['biden_v'])]
dfn

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04T07:47:41Z,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,113744.0,68815.120,40037.888,0.0,4890.992,68815.120,40037.888,0,4890.992,1.718750
2020-11-04T08:57:55Z,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,18731.0,11500.834,6499.657,0.0,730.509,81339.650,45968.825,0,5166.525,1.769452
2020-11-04T18:34:58Z,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,39556.0,24880.724,13053.480,0.0,1621.796,108207.499,56770.230,0,7053.271,1.906061
2020-11-09T19:09:13Z,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,18939.0,11780.058,6363.504,0.0,795.438,118783.340,64165.920,0,8020.740,1.851190
2020-11-11T02:46:15Z,5,240234,75,edison,alaska,367000,163387,300495,318608,0.580,...,49264.0,28573.120,18769.584,0.0,1921.296,139335.720,91529.154,0,9369.126,1.522310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-04T07:16:15Z,8942,232266,80,edison,wyoming,280000,174419,249061,255849,0.715,...,14677.0,10494.055,3683.927,0.0,499.018,166070.190,58298.766,0,7897.044,2.848606
2020-11-04T08:29:35Z,8943,268852,93,edison,wyoming,280000,174419,249061,255849,0.697,...,36586.0,25500.442,9805.048,0.0,1280.510,187389.844,72052.336,0,9409.820,2.600746
2020-11-04T08:45:32Z,8944,271429,94,edison,wyoming,280000,174419,249061,255849,0.698,...,2577.0,1798.746,688.059,0.0,90.195,189457.442,72471.543,0,9500.015,2.614232


In [201]:
# Need to find the growing increments of votes for each time by each state
filt = (dfn['state'] == 'arizona')
dfs = dfn[filt]
dfs[['votes','votes_diff','trump_v','biden_v','r_to_d']]

,votes,votes_diff,trump_v,biden_v,r_to_d
timestamp,,,,,
2020-11-04T03:03:36Z,104866,104866.0,52957.330,50335.680,1.052083
2020-11-04T03:05:47Z,730299,625433.0,263307.293,353995.078,0.743816
2020-11-04T03:06:25Z,2203485,1473186.0,643782.282,810252.300,0.794545
2020-11-04T03:08:46Z,2234553,31068.0,13607.784,17056.332,0.797814
2020-11-04T03:10:54Z,2304343,69790.0,31126.340,37756.390,0.824399
...,...,...,...,...,...
2020-11-11T00:37:43Z,3342739,1226.0,600.740,605.644,0.991903
2020-11-11T00:48:42Z,3342741,2.0,0.980,0.988,0.991903
2020-11-11T00:57:45Z,3343125,384.0,188.160,189.696,0.991903


In [204]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfs.index,y=dfs['vote_share_dem'],name='Dem'))
fig.add_trace(go.Scatter(x=dfs.index, y=dfs['vote_share_rep'],name='Rep'))
fig.show()

In [213]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfn.index,y=dfn['r_to_d'],name='Ratio',mode='markers',))

fig = px.scatter(dfn,x=dfn.index,y=dfn['r_to_d'],color=dfn['state'])
pio.write_html(fig, file=f'ratio.html', auto_open=True)
fig.show()

In [ ]:
states = df['state'].unique()
filt = (df['state'] == 'illinois')
df[filt]

In [ ]:
fig = px.scatter_3d(df, x=df.index, y=df['state'], z=df['trump_v'],color=df['state'])
fig.show()
pio.write_html(fig, file=f'3d.html', auto_open=True)

